In [65]:
import pandas as pd
pd.set_option("display.precision", 2)
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime as dt
import itertools
from tqdm import tqdm
%matplotlib inline
from sklearn.metrics import accuracy_score,log_loss
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

In [66]:
train = pd.read_csv("Train.csv",parse_dates=['Date'])
test = pd.read_csv("Test.csv",parse_dates=['Date'])

In [67]:
train

,Date,Season,Match_ID,Game_ID,Home Team,Away Team,Score
0,2017-02-15,1,1.0,ID_KAG4KAE9,Antennae,Andromeda,Away win
1,2016-09-14,1,12.0,ID_T7H1XN5Z,Andromeda,Antennae,Draw
2,2016-12-12,1,13.0,ID_O1P9FGN4,Andromeda,Butterfly,Away win
3,2017-01-25,1,14.0,ID_TR5M4KQV,Andromeda,Cigar,Away win
4,2016-08-24,1,15.0,ID_EOIECRE2,Andromeda,Circinus,Home Win
...,...,...,...,...,...,...,...
407,2016-08-28,1,59.0,ID_PREVA6HZ,Sculptor,Medusa Merger,Away win
408,2017-02-19,1,151.0,ID_9DJ219JY,Medusa Merger,Sculptor,Home Win
409,2016-10-23,1,154.0,ID_A1PN23J6,Medusa Merger,Sombrero,Home Win
410,2017-08-16,1,171.0,ID_XFQO60PZ,Milky Way,Sculptor,Draw


In [68]:
le=LabelEncoder()
train["Score"] = le.fit_transform(train["Score"])
score_mapping = dict(zip(le.classes_, range(len(le.classes_))))

In [69]:
score_mapping

{'Away win': 0, 'Draw': 1, 'Home Win': 2}

In [70]:
train.sort_values(by=['Date'],inplace=True)

In [71]:
test.sort_values(by=['Date'],inplace=True)

In [72]:
train["Train"] = 1
test["Train"]= 0

train_test = pd.concat([train, test])

In [73]:
train_test

,Date,Season,Match_ID,Game_ID,Home Team,Away Team,Score,Train
63,2016-08-03,1,2.0,ID_Z71PIKP3,Antennae,Butterfly,0.0,1
389,2016-08-03,1,118.0,ID_J1010ZTT,Cosmos Redshift 7,Sculptor,2.0,1
324,2016-08-03,1,77.0,ID_IDJ1L6GL,Circinus,Fireworks,1.0,1
120,2016-08-03,1,22.0,ID_T12UUJG7,Backward,Cartwheel,1.0,1
383,2016-08-03,1,158.0,ID_WT3PHABY,Medusa Merger,Milky Way,2.0,1
...,...,...,...,...,...,...,...,...
115,2019-05-21,3,211.0,ID_H6H66KH2,Cigar,Tadpole,NaN,0
217,2019-05-21,3,300.0,ID_QR4DQRG1,Needle,Malin 1,NaN,0
37,2019-05-23,3,214.0,ID_QKWD8147,Whirlpool,Medusa Merger,NaN,0
73,2019-05-24,3,216.0,ID_DLPZZFIJ,Antennae,Andromeda,NaN,0


In [74]:
test_game_statistics = pd.read_csv("test_game_statistics.csv")
train_game_statistics = pd.read_csv("train_game_statistics.csv")

### for instance we will drop some nan columns

In [75]:
train_game_statistics.drop(columns=["next_player","next_action","next_x","next_y","event_id","next_team","next_event_id","xt_value"],inplace=True)

In [76]:
test_game_statistics.drop(columns=["next_player","next_x","next_y","event_id","next_team","next_event_id","xt_value"],inplace=True)

In [77]:
game_stats = pd.concat([train_game_statistics, test_game_statistics])

In [78]:
all_data = game_stats.merge(train_test.drop(columns="Season") , on="Game_ID")
all_data=all_data.sort_values(by=["Date","Home Team","id"],ascending=[True,True,True]).reset_index(drop=True)
game_mapping={k: v for v, k in enumerate(all_data["Game_ID"].unique())}
all_data["Game_ID"]=all_data["Game_ID"].apply(lambda x: game_mapping[x])
train_test["Game_ID"]=train_test["Game_ID"].apply(lambda x: game_mapping[x])
#le = preprocessing.LabelEncoder()
#all_data["Game_ID"] = le.fit_transform(all_data["Game_ID"])
#game_mapping = dict(zip(le.classes_, range(len(le.classes_))))
#all_data=all_data.sort_values(by=["Game_ID"],ascending=True).reset_index(drop=True)

### halves

In [79]:
le = preprocessing.LabelEncoder()
all_data["Half"] = le.fit_transform(all_data["Half"])

### X and Y

In [80]:
all_data.loc[(all_data["Team"]==all_data["Away Team"]),"X"]*=-1
all_data.loc[(all_data["Team"]==all_data["Away Team"]),"Y"]*=-1

all_data.loc[(all_data["Team"]==all_data["Home Team"]),"X"]-=52.5
all_data.loc[(all_data["Team"]==all_data["Home Team"]),"Y"]-=34

all_data.loc[(all_data["Team"]==all_data["Away Team"]),"X"]+=52.5
all_data.loc[(all_data["Team"]==all_data["Away Team"]),"Y"]+=34

In [81]:
all_data1=all_data[all_data["Half"]==0]
all_data2=all_data[all_data["Half"]==1]

# accurate and innaccurate passes

In [82]:
train_test=train_test.sort_values("Game_ID").reset_index(drop=True)

In [83]:
#whole match

#home
train_test[["H_acc","H_inacc"]]=all_data[all_data["Team"]==all_data["Home Team"]].groupby("Game_ID").sum()[["Accurate passes","Inaccurate passes"]]
train_test["H_acc_inacc"]=train_test["H_acc"]/(train_test["H_inacc"])

#away
train_test[["A_acc","A_inacc"]]=all_data[all_data["Team"]==all_data["Away Team"]].groupby("Game_ID").sum()[["Accurate passes","Inaccurate passes"]]
train_test["A_acc_inacc"]=train_test["A_acc"]/(train_test["A_inacc"])


In [84]:
#1st half

#home
train_test[["H_acc1","H_inacc1"]]=all_data1[all_data1["Team"]==all_data1["Home Team"]].groupby("Game_ID").sum()[["Accurate passes","Inaccurate passes"]]
train_test["H_acc_inacc1"]=train_test["H_acc1"]/(train_test["H_inacc1"])

#away
train_test[["H_acc2","H_inacc2"]]=all_data2[all_data2["Team"]==all_data2["Home Team"]].groupby("Game_ID").sum()[["Accurate passes","Inaccurate passes"]]
train_test["H_acc_inacc2"]=train_test["H_acc2"]/(train_test["H_inacc2"])


In [85]:
#2nd half

#home
train_test[["A_acc1","A_inacc1"]]=all_data1[all_data1["Team"]==all_data1["Away Team"]].groupby("Game_ID").sum()[["Accurate passes","Inaccurate passes"]]
train_test["A_acc_inacc1"]=train_test["A_acc1"]/(train_test["A_inacc1"])

#away
train_test[["A_acc2","A_inacc2"]]=all_data2[all_data2["Team"]==all_data2["Away Team"]].groupby("Game_ID").sum()[["Accurate passes","Inaccurate passes"]]
train_test["A_acc_inacc2"]=train_test["A_acc2"]/(train_test["A_inacc2"])


In [86]:
# the same thing with dangerous accurate and incaccurate passes from both sides

In [87]:
dangerous1= all_data[(all_data["X"]>30) & ((all_data["Y"]<15)&(all_data["Y"]>-15))]
dangerous2= all_data[(all_data["X"]<-30) & ((all_data["Y"]<15)&(all_data["Y"]>-15))]


In [88]:
train_test["D_H_acc"]=dangerous1[(dangerous1["Team"]==dangerous1["Home Team"])].groupby("Game_ID").sum()["Accurate passes"]
train_test["D_H_inacc"]=dangerous2[(dangerous2["Team"]==dangerous2["Home Team"])].groupby("Game_ID").sum()["Inaccurate passes"]

train_test["D_A_acc"]=dangerous2[dangerous2["Team"]==dangerous2["Away Team"]].groupby("Game_ID").sum()["Accurate passes"]
train_test["D_A_inacc"]=dangerous1[dangerous1["Team"]==dangerous1["Away Team"]].groupby("Game_ID").sum()["Inaccurate passes"]


### action performed
(since it s so difficult to calculate possession for each team)

In [89]:
#whole match
train_test["H_action"]=all_data[all_data["Team"]==all_data["Home Team"]].groupby("Game_ID")["Player_ID"].count()
train_test["A_action"]=all_data[all_data["Team"]==all_data["Away Team"]].groupby("Game_ID")["Player_ID"].count()

#1st half

#home
train_test["H_action1"]=all_data1[all_data1["Team"]==all_data1["Home Team"]].groupby("Game_ID")["Player_ID"].count()

#away
train_test["A_action1"]=all_data1[all_data1["Team"]==all_data1["Away Team"]].groupby("Game_ID")["Player_ID"].count()

#2nd half

#home
train_test["H_action2"]=all_data2[all_data2["Team"]==all_data2["Home Team"]].groupby("Game_ID")["Player_ID"].count()


#away
train_test["A_action2"]=all_data2[all_data2["Team"]==all_data2["Away Team"]].groupby("Game_ID")["Player_ID"].count()


In [90]:
train_test["D_H_action"]=dangerous1[(dangerous1["Team"]==dangerous1["Home Team"])].groupby("Game_ID")["Player_ID"].count()

train_test["D_A_action"]=dangerous2[dangerous2["Team"]==dangerous2["Away Team"]].groupby("Game_ID")["Player_ID"].count()

### one of the opponents is new to the league

In [91]:
s1_t= set(train_test[train_test["Season"]==1]["Home Team"].unique()).union(set(train_test[train_test["Season"]==1]["Away Team"].unique()))
s2_t= set(train_test[train_test["Season"]==2]["Home Team"].unique()).union(set(train_test[train_test["Season"]==2]["Away Team"].unique()))
s3_t= set(train_test[train_test["Season"]==3]["Home Team"].unique()).union(set(train_test[train_test["Season"]==3]["Away Team"].unique()))


In [92]:
train_test["new"]=0
#train_test.loc[train_test["Season"]==1,"new"]=0

In [93]:
for team in s2_t-s1_t:
    train_test.loc[(train_test["Home Team"]==team)&(train_test["Season"]==2),"new"]=1
    train_test.loc[(train_test["Away Team"]==team)&(train_test["Season"]==2),"new"]=2
    
for team in s3_t-s2_t:
    train_test.loc[(train_test["Home Team"]==team)&(train_test["Season"]==3),"new"]=1
    train_test.loc[(train_test["Away Team"]==team)&(train_test["Season"]==3),"new"]=2

for team in s3_t-s2_t-s1_t:
    train_test.loc[(train_test["Home Team"]==team)&(train_test["Season"]==3),"new"]=2
    train_test.loc[(train_test["Away Team"]==team)&(train_test["Season"]==3),"new"]=3

### managers

In [94]:
train_test["Home Manager"]=all_data[all_data["Team"]==all_data["Home Team"]].drop_duplicates("Game_ID")["Manager"].reset_index(drop=True)
train_test["Away Manager"]=all_data[all_data["Team"]==all_data["Away Team"]].drop_duplicates("Game_ID")["Manager"].reset_index(drop=True)


In [95]:
train_test

,Date,Season,Match_ID,Game_ID,Home Team,Away Team,Score,Train,H_acc,H_inacc,H_acc_inacc,A_acc,A_inacc,A_acc_inacc,H_acc1,H_inacc1,H_acc_inacc1,H_acc2,H_inacc2,H_acc_inacc2,A_acc1,A_inacc1,A_acc_inacc1,A_acc2,A_inacc2,A_acc_inacc2,D_H_acc,D_H_inacc,D_A_acc,D_A_inacc,H_action,A_action,H_action1,A_action1,H_action2,A_action2,D_H_action,D_A_action,new,Home Manager,Away Manager
0,2016-08-03,1,2.0,0,Antennae,Butterfly,0.0,1,463.0,88.0,5.26,315.0,77.0,4.09,260.0,36.0,7.22,203.0,52.0,3.90,167.0,36.0,4.64,148.0,41.0,3.61,12.0,2.0,6.0,7.0,2361,1803,1180.0,916.0,1181.0,887.0,145.0,109.0,0,Devix,Dante
1,2016-08-03,1,22.0,1,Backward,Cartwheel,1.0,1,255.0,109.0,2.34,450.0,97.0,4.64,149.0,50.0,2.98,106.0,59.0,1.80,183.0,38.0,4.82,267.0,59.0,4.53,4.0,5.0,3.0,5.0,1903,2309,982.0,970.0,921.0,1339.0,135.0,77.0,0,Osamu,Elrick
2,2016-08-03,1,77.0,2,Circinus,Fireworks,1.0,1,626.0,102.0,6.14,370.0,88.0,4.20,332.0,34.0,9.76,294.0,68.0,4.32,208.0,39.0,5.33,162.0,49.0,3.31,13.0,1.0,0.0,14.0,2763,1878,1312.0,934.0,1451.0,944.0,168.0,19.0,0,Kouta,Jiraya
3,2016-08-03,1,118.0,3,Cosmos Redshift 7,Sculptor,2.0,1,550.0,78.0,7.05,339.0,73.0,4.64,253.0,28.0,9.04,297.0,50.0,5.94,179.0,36.0,4.97,160.0,37.0,4.32,9.0,6.0,9.0,3.0,2414,1773,1087.0,904.0,1327.0,869.0,116.0,125.0,0,Bolt,Ryoichi
4,2016-08-03,1,158.0,4,Medusa Merger,Milky Way,2.0,1,499.0,81.0,6.16,280.0,84.0,3.33,268.0,35.0,7.66,231.0,46.0,5.02,133.0,37.0,3.59,147.0,47.0,3.13,9.0,8.0,6.0,5.0,2290,1687,1195.0,790.0,1095.0,897.0,141.0,128.0,0,Levi,Masashi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,2019-05-21,3,213.0,641,Fireworks,Cartwheel,NaN,0,386.0,93.0,4.15,346.0,98.0,3.53,215.0,49.0,4.39,171.0,44.0,3.89,164.0,53.0,3.09,182.0,45.0,4.04,3.0,5.0,5.0,7.0,1947,1882,1042.0,927.0,905.0,955.0,67.0,100.0,0,Orgulas,Isamu
642,2019-05-21,3,300.0,642,Needle,Malin 1,NaN,0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,4,4,2.0,2.0,2.0,2.0,NaN,NaN,3,NaN,NaN
643,2019-05-23,3,214.0,643,Whirlpool,Medusa Merger,NaN,0,311.0,71.0,4.38,320.0,67.0,4.78,130.0,30.0,4.33,181.0,41.0,4.41,169.0,36.0,4.69,151.0,31.0,4.87,12.0,3.0,2.0,11.0,1607,1498,677.0,743.0,930.0,755.0,159.0,75.0,2,Tatsuya,Akuma/Inuyasha
644,2019-05-24,3,216.0,644,Antennae,Andromeda,NaN,0,385.0,99.0,3.89,329.0,71.0,4.63,178.0,42.0,4.24,207.0,57.0,3.63,218.0,38.0,5.74,111.0,33.0,3.36,3.0,7.0,7.0,7.0,1908,1803,843.0,999.0,1065.0,804.0,89.0,124.0,0,Nikushimi,Tarkas/Adelard


In [96]:
df = pd.get_dummies(train_test, columns = ["Home Team","Away Team","Home Manager","Away Manager"])

In [97]:
def manager_stats(df,season):
    point_score_home={0:0,1:1,2:3}
    point_score_away={0:3,1:1,2:0}
    managers=set(df.loc[df["Season"]==season]["Home Manager"].unique()).union(set(df.loc[df["Season"]==season]["Away Manager"].unique()))
    
    stats=pd.DataFrame(managers,columns=["Manager"])
    stats["Played"]=0.0
    stats["Points"]=0.0
    stats["ratio"]=0.0
    for manager in managers:
        h=df[(df["Home Manager"] == manager ) & (df["Season"] == season)]["Score"]
        a=df[(df["Away Manager"] == manager ) & (df["Season"] == season)]["Score"]
        stats.loc[stats["Manager"]==manager,"Points"]=a.apply(lambda x: point_score_away[x]).sum()+h.apply(lambda x: point_score_home[x]).sum()
        stats.loc[stats["Manager"]==manager,"Played"]=len(a)+len(h)
        if stats.loc[stats["Manager"]==manager,"Played"].values[0]>0:
            stats.loc[stats["Manager"]==manager,"ratio"]=(stats.loc[stats["Manager"]==manager,"Points"]+1)/np.log(stats.loc[stats["Manager"]==manager,"Played"]+1)
    return stats
m_s1=manager_stats(train_test[train_test["Train"]==1],1).fillna(0).sort_values(by=['ratio'], ascending=False)
m_s2=manager_stats(train_test[train_test["Train"]==1],2).fillna(0).sort_values(by=['ratio'], ascending=False)

In [98]:
a=dict(m_s1.set_index("Manager")["ratio"])
b=dict(m_s2.set_index("Manager")["ratio"])
c=dict()
for manager in train_test[train_test["Train"]==1]["Home Manager"].unique():
    if manager in a.keys() and manager in b.keys():
        c[manager]=max(a[manager],b[manager])
    elif manager not in a.keys():
        c[manager]=b[manager]
    else:
        c[manager]=a[manager]

In [99]:
train_test.loc[train_test["Season"]==1,"Home Manager"]=train_test.loc[train_test["Season"]==1,"Home Manager"].apply(lambda x : a[x])
train_test.loc[train_test["Season"]==2,"Home Manager"]=train_test.loc[train_test["Season"]==2,"Home Manager"].apply(lambda x : b[x])
train_test.loc[train_test["Season"]==3,"Home Manager"]=train_test.loc[train_test["Season"]==3,"Home Manager"].apply(lambda x : c[x] if x in c else np.nan)

train_test.loc[train_test["Season"]==1,"Away Manager"]=train_test.loc[train_test["Season"]==1,"Away Manager"].apply(lambda x : a[x])
train_test.loc[train_test["Season"]==2,"Away Manager"]=train_test.loc[train_test["Season"]==2,"Away Manager"].apply(lambda x : b[x])
train_test.loc[train_test["Season"]==3,"Away Manager"]=train_test.loc[train_test["Season"]==3,"Away Manager"].apply(lambda x : c[x] if x in c else np.nan)

In [100]:
#some managers are new in season 3. There is no way to attribute an effective score to them since no info were provided
# my approach is give them the average score of the two predecessors managers of the club. 
# For the managers of new clubs that have not been neither in s1 or s2... 
manager="Home Manager"

for team in set(train_test.loc[pd.isna(train_test[manager])]["Home Team"].unique()).union(set(train_test.loc[pd.isna(train_test[manager])]["Away Team"].unique())):
    s1=0
    s2=0
    if team in train_test.loc[train_test["Season"]==1]["Home Team"].unique() and team in train_test.loc[train_test["Season"]==2]["Home Team"].unique()  :
        s1=0.5*train_test.loc[(train_test["Home Team"]==team) & (train_test["Season"]==1)][manager].mean()
        s2=0.5*train_test.loc[(train_test["Home Team"]==team) & (train_test["Season"]==2)][manager].mean()

    elif team in train_test.loc[train_test["Season"]==1]["Home Team"].unique():
        s1=train_test.loc[(train_test["Home Team"]==team) & (train_test["Season"]==1)][manager].mean()
        

    elif team in train_test.loc[train_test["Season"]==2]["Home Team"].unique():
        s2=train_test.loc[(train_test["Home Team"]==team) & (train_test["Season"]==2)][manager].mean()
        

    else:
        continue

    train_test.loc[(pd.isna(train_test[manager]))&(train_test["Home Team"]==team),manager]=s1+s2


        

In [101]:
manager="Away Manager"

for team in set(train_test.loc[pd.isna(train_test[manager])]["Away Team"].unique()).union(set(train_test.loc[pd.isna(train_test[manager])]["Away Team"].unique())):
    s1=0
    s2=0
    if team in train_test.loc[train_test["Season"]==1]["Away Team"].unique() and team in train_test.loc[train_test["Season"]==2]["Away Team"].unique()  :
        s1=0.5*train_test.loc[(train_test["Away Team"]==team) & (train_test["Season"]==1)][manager].mean()
        s2=0.5*train_test.loc[(train_test["Away Team"]==team) & (train_test["Season"]==2)][manager].mean()

    elif team in train_test.loc[train_test["Season"]==1]["Away Team"].unique():
        s1=train_test.loc[(train_test["Away Team"]==team) & (train_test["Season"]==1)][manager].mean()
        

    elif team in train_test.loc[train_test["Season"]==2]["Away Team"].unique():
        s2=train_test.loc[(train_test["Away Team"]==team) & (train_test["Season"]==2)][manager].mean()
        

    else:
        continue

    train_test.loc[(pd.isna(train_test[manager]))&(train_test["Away Team"]==team),manager]=s1+s2


        

In [102]:
train_test.loc[pd.isna(train_test["Home Manager"])]["Home Team"].unique()

array(['Malin 1', 'Needle', 'Whirlpool'], dtype=object)

**as we can see there are still 3 clubs ( usually every season 2 or 3 clubs get promoted to the higher division)**

In [103]:
#the best we can do is assign the min value to each of them

train_test.loc[pd.isna(train_test["Home Manager"]),"Home Manager"] = \
train_test.loc[pd.isna(train_test["Home Manager"])]["Home Manager"].fillna(train_test.loc[~pd.isna(train_test["Home Manager"])]["Home Manager"].min())

In [104]:
train_test.loc[pd.isna(train_test["Away Manager"]),"Away Manager"] = \
train_test.loc[pd.isna(train_test["Away Manager"])]["Away Manager"].fillna(train_test.loc[~pd.isna(train_test["Away Manager"])]["Away Manager"].min())

### Danger zone

In [105]:
# How many time a team has been in the danger zone

train_test["H_Danger"]=dangerous1[(dangerous1["Team"]==dangerous1["Home Team"])].groupby("Game_ID")["Train"].count()

train_test["A_Danger"]=dangerous2[(dangerous2["Team"]==dangerous2["Away Team"])].groupby("Game_ID")["Train"].count()



### Teams

In [106]:
def stats(df,season):
    point_score_home={0:0,1:1,2:3}
    point_score_away={0:3,1:1,2:0}
    teams=df["Home Team"].unique()
    stats=pd.DataFrame(teams,columns=["Teams"])
    stats["Played"]=0
    stats["Points"]=0
    stats["ratio"]=0
    for team in teams:
        h=df[ (df["Home Team"]==team) & (df["Season"]==season)]["Score"]
        a=df[ (df["Away Team"]==team) & (df["Season"]==season)]["Score"]

        stats.loc[stats["Teams"]==team,"Points"]=h.apply(lambda x: point_score_home[x]).sum()+a.apply(lambda x: point_score_away[x]).sum()
        stats.loc[stats["Teams"]==team,"Played"]=len(a)+len(h)
        try:
            stats.loc[stats["Teams"]==team,"ratio"]=stats.loc[stats["Teams"]==team,"Points"]/stats.loc[stats["Teams"]==team,"Played"]
        except:
            pass
    return stats
stats_s1=stats(train_test[train_test["Train"]==1],1).fillna(0).sort_values(by=['ratio'], ascending=False)
stats_s2=stats(train_test[train_test["Train"]==1],2).fillna(0).sort_values(by=['ratio'], ascending=False)

In [107]:
# now we will be replacing teams name by the ratio according to each season
# for the test aka season 3 the ratio would be 40% ratio s1 + 60% ratio s2
a=dict(stats_s1.set_index("Teams")["ratio"])
b=dict(stats_s2.set_index("Teams")["ratio"])
from collections import Counter
a1={key: value * 0.4 for key, value in a.items()}
b1={key: value * 0.6 for key, value in b.items()}
c=dict(Counter(a1)+Counter(b1))

In [108]:
cols=["Home Team","Away Team"]
for col in cols:
    train_test.loc[train_test["Season"]==1,col]=train_test.loc[train_test["Season"]==1,col].apply(lambda x : a[x] if x in a else 0)

for col in cols:
    train_test.loc[train_test["Season"]==2,col]=train_test.loc[train_test["Season"]==2,col].apply(lambda x : b[x] if x in b else 0)

for col in cols:
    train_test.loc[train_test["Season"]==3,col]=train_test.loc[train_test["Season"]==3,col].apply(lambda x : c[x] if x in c else 0)        


In [109]:
train_test.drop("Date",axis=1,inplace=True)

In [110]:
train_test=train_test.astype(np.float)

In [111]:
train_test=pd.concat([train_test,df.iloc[:,37:]],axis=1)

## Model

In [112]:
X = train_test[train_test["Train"]==1].drop(["Train","Score"], axis=1)
y = train_test[train_test["Train"]==1]["Score"]
print(X.columns[:45])

Index(['Season', 'Match_ID', 'Game_ID', 'Home Team', 'Away Team', 'H_acc',
       'H_inacc', 'H_acc_inacc', 'A_acc', 'A_inacc', 'A_acc_inacc', 'H_acc1',
       'H_inacc1', 'H_acc_inacc1', 'H_acc2', 'H_inacc2', 'H_acc_inacc2',
       'A_acc1', 'A_inacc1', 'A_acc_inacc1', 'A_acc2', 'A_inacc2',
       'A_acc_inacc2', 'D_H_acc', 'D_H_inacc', 'D_A_acc', 'D_A_inacc',
       'H_action', 'A_action', 'H_action1', 'A_action1', 'H_action2',
       'A_action2', 'D_H_action', 'D_A_action', 'new', 'Home Manager',
       'Away Manager', 'H_Danger', 'A_Danger', 'Home Team_Andromeda',
       'Home Team_Antennae', 'Home Team_Backward', 'Home Team_Butterfly',
       'Home Team_Cartwheel'],
      dtype='object')


In [113]:
Test=train_test[train_test["Train"]==0].drop(["Train","Score"], axis=1)

In [115]:
X_train = X.loc[X["Season"]==1]
X_test = X.loc[X["Season"]==2]
y_train = y.iloc[:X_train.shape[0]]
y_test = y.iloc[X_train.shape[0]:]


In [116]:
X.fillna(0)
Test.fillna(0)

,Season,Match_ID,Game_ID,Home Team,Away Team,H_acc,H_inacc,H_acc_inacc,A_acc,A_inacc,A_acc_inacc,H_acc1,H_inacc1,H_acc_inacc1,H_acc2,H_inacc2,H_acc_inacc2,A_acc1,A_inacc1,A_acc_inacc1,A_acc2,A_inacc2,A_acc_inacc2,D_H_acc,D_H_inacc,D_A_acc,D_A_inacc,H_action,A_action,H_action1,A_action1,H_action2,A_action2,D_H_action,D_A_action,new,Home Manager,Away Manager,H_Danger,A_Danger,Home Team_Andromeda,Home Team_Antennae,Home Team_Backward,Home Team_Butterfly,Home Team_Cartwheel,Home Team_Cigar,Home Team_Circinus,Home Team_Coma Pinwheel,Home Team_Comet,Home Team_Cosmos Redshift 7,Home Team_Eye of Sauron,Home Team_Fireworks,Home Team_Malin 1,Home Team_Medusa Merger,Home Team_Milky Way,Home Team_Needle,Home Team_Sculptor,Home Team_Sombrero,Home Team_Sunflower,Home Team_Tadpole,Home Team_Triangulum,Home Team_Whirlpool,Away Team_Andromeda,Away Team_Antennae,Away Team_Backward,Away Team_Butterfly,Away Team_Cartwheel,Away Team_Cigar,Away Team_Circinus,Away Team_Coma Pinwheel,Away Team_Comet,Away Team_Cosmos Redshift 7,Away Team_Eye of Sauron,Away Team_Fireworks,Away Team_Malin 1,Away Team_Medusa Merger,Away Team_Milky Way,Away Team_Needle,Away Team_Sculptor,Away Team_Sombrero,Away Team_Sunflower,Away Team_Tadpole,Away Team_Triangulum,Away Team_Whirlpool,Home Manager_Adric,Home Manager_Akui,Home Manager_Akuma/Inuyasha,Home Manager_Alex,Home Manager_Alphonse,Home Manager_Andwise/Hob,Home Manager_Anson,Home Manager_Aoiro,Home Manager_Ash,Home Manager_Balbo,Home Manager_Barner,Home Manager_Beau,Home Manager_Bill,Home Manager_Bodo/Drogo,Home Manager_Bolt,Home Manager_Brock,Home Manager_Cottar,Home Manager_Daiki,Home Manager_Dante,Home Manager_Devix,Home Manager_Dudo/Eder,Home Manager_Elrick,Home Manager_Frodo,Home Manager_Haldon,Home Manager_Hideaki,Home Manager_Hideo,Home Manager_Hikaru,Home Manager_Hornblower,Home Manager_Isamu,Home Manager_Jiraya,Home Manager_Joyas,Home Manager_Karatachi,Home Manager_Kepli,Home Manager_Kouki,Home Manager_Kouta,Home Manager_Kuro,Home Manager_Largo,Home Manager_Levi,Home Manager_Masashi,Home Manager_Megumi,Home Manager_Nikushimi,Home Manager_Noburu,Home Manager_Olo,Home Manager_Orgulas,Home Manager_Osamu,Home Manager_Otho,Home Manager_Ronald,Home Manager_Rorimac,Home Manager_Roy,Home Manager_Rufus,Home Manager_Ryoichi,Home Manager_Ryuu,Home Manager_Seiji,Home Manager_Shinichi,Home Manager_Tanjiro,Home Manager_Tarkas/Adelard,Home Manager_Tatsuya,Home Manager_Tenzin,Home Manager_Valkas/Gokin,Home Manager_Yami,Home Manager_Youkai,Home Manager_Zankoku/Garret,Away Manager_Adelard,Away Manager_Adric,Away Manager_Akui,Away Manager_Akuma/Inuyasha,Away Manager_Alex,Away Manager_Alphonse,Away Manager_Andwise/Hob,Away Manager_Aoiro,Away Manager_Ash,Away Manager_Astro,Away Manager_Balbo,Away Manager_Barner,Away Manager_Bill,Away Manager_Bodo/Drogo,Away Manager_Bolt,Away Manager_Brock,Away Manager_Cottar,Away Manager_Daiki,Away Manager_Dante,Away Manager_Devix,Away Manager_Dudo/Eder,Away Manager_Elrick,Away Manager_Frodo,Away Manager_Haldon,Away Manager_Hideaki,Away Manager_Hideo,Away Manager_Hikaru,Away Manager_Hornblower,Away Manager_Isamu,Away Manager_Jiraya,Away Manager_Joyas,Away Manager_Karatachi,Away Manager_Kepli,Away Manager_Kouki,Away Manager_Kouta,Away Manager_Largo,Away Manager_Levi,Away Manager_Masashi,Away Manager_Megumi,Away Manager_Nikushimi,Away Manager_Noburu,Away Manager_Osamu,Away Manager_Otho,Away Manager_Rorimac,Away Manager_Roy,Away Manager_Rufus,Away Manager_Ryoichi,Away Manager_Ryuu,Away Manager_Seiji,Away Manager_Shinichi,Away Manager_Tanjiro,Away Manager_Tarkas/Adelard,Away Manager_Tatsuya,Away Manager_Tenzin,Away Manager_Valkas/Gokin,Away Manager_Yami,Away Manager_Youkai,Away Manager_Zankoku/Garret
412,3.0,3.0,412.0,1.40,1.20,368.0,98.0,3.76,325.0,82.0,3.96,228.0,50.0,4.56,140.0,48.0,2.92,167.0,28.0,5.96,158.0,54.0,2.93,5.0,12.0,13.0,4.0,1961.0,1895.0,1133.0,917.0,828.0,978.0,91.0,165.0,0.0,7.97,16.89,91.0,165.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,

In [117]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() 

cols=[ 'Home Team', 'Away Team',
       'H_acc', 'H_inacc', 'H_acc_inacc', 'A_acc', 'A_inacc', 'A_acc_inacc',
       'H_acc1', 'H_inacc1', 'H_acc_inacc1', 'H_acc2', 'H_inacc2',
       'H_acc_inacc2', 'A_acc1', 'A_inacc1', 'A_acc_inacc1', 'A_acc2',
       'A_inacc2', 'A_acc_inacc2', 'D_H_acc', 'D_H_inacc', 'D_A_acc',
       'D_A_inacc', 'H_action', 'A_action', 'H_action1', 'A_action1',
       'H_action2', 'A_action2', 'D_H_action', 'D_A_action',
       'Home Manager', 'Away Manager', 'H_Danger', 'A_Danger']
X_scaled = scaler.fit_transform(X) 
X_scaled=pd.DataFrame(X_scaled,columns=X.columns)
scaler = MinMaxScaler() 
Test_scaled = scaler.fit_transform(Test) 
Test_scaled=pd.DataFrame(Test_scaled,columns=Test.columns)

#X.loc[:,cols] = X_scaled[cols]
#Test.loc[:,cols] = Test_scaled[cols]

 



In [118]:
X

,Season,Match_ID,Game_ID,Home Team,Away Team,H_acc,H_inacc,H_acc_inacc,A_acc,A_inacc,A_acc_inacc,H_acc1,H_inacc1,H_acc_inacc1,H_acc2,H_inacc2,H_acc_inacc2,A_acc1,A_inacc1,A_acc_inacc1,A_acc2,A_inacc2,A_acc_inacc2,D_H_acc,D_H_inacc,D_A_acc,D_A_inacc,H_action,A_action,H_action1,A_action1,H_action2,A_action2,D_H_action,D_A_action,new,Home Manager,Away Manager,H_Danger,A_Danger,Home Team_Andromeda,Home Team_Antennae,Home Team_Backward,Home Team_Butterfly,Home Team_Cartwheel,Home Team_Cigar,Home Team_Circinus,Home Team_Coma Pinwheel,Home Team_Comet,Home Team_Cosmos Redshift 7,Home Team_Eye of Sauron,Home Team_Fireworks,Home Team_Malin 1,Home Team_Medusa Merger,Home Team_Milky Way,Home Team_Needle,Home Team_Sculptor,Home Team_Sombrero,Home Team_Sunflower,Home Team_Tadpole,Home Team_Triangulum,Home Team_Whirlpool,Away Team_Andromeda,Away Team_Antennae,Away Team_Backward,Away Team_Butterfly,Away Team_Cartwheel,Away Team_Cigar,Away Team_Circinus,Away Team_Coma Pinwheel,Away Team_Comet,Away Team_Cosmos Redshift 7,Away Team_Eye of Sauron,Away Team_Fireworks,Away Team_Malin 1,Away Team_Medusa Merger,Away Team_Milky Way,Away Team_Needle,Away Team_Sculptor,Away Team_Sombrero,Away Team_Sunflower,Away Team_Tadpole,Away Team_Triangulum,Away Team_Whirlpool,Home Manager_Adric,Home Manager_Akui,Home Manager_Akuma/Inuyasha,Home Manager_Alex,Home Manager_Alphonse,Home Manager_Andwise/Hob,Home Manager_Anson,Home Manager_Aoiro,Home Manager_Ash,Home Manager_Balbo,Home Manager_Barner,Home Manager_Beau,Home Manager_Bill,Home Manager_Bodo/Drogo,Home Manager_Bolt,Home Manager_Brock,Home Manager_Cottar,Home Manager_Daiki,Home Manager_Dante,Home Manager_Devix,Home Manager_Dudo/Eder,Home Manager_Elrick,Home Manager_Frodo,Home Manager_Haldon,Home Manager_Hideaki,Home Manager_Hideo,Home Manager_Hikaru,Home Manager_Hornblower,Home Manager_Isamu,Home Manager_Jiraya,Home Manager_Joyas,Home Manager_Karatachi,Home Manager_Kepli,Home Manager_Kouki,Home Manager_Kouta,Home Manager_Kuro,Home Manager_Largo,Home Manager_Levi,Home Manager_Masashi,Home Manager_Megumi,Home Manager_Nikushimi,Home Manager_Noburu,Home Manager_Olo,Home Manager_Orgulas,Home Manager_Osamu,Home Manager_Otho,Home Manager_Ronald,Home Manager_Rorimac,Home Manager_Roy,Home Manager_Rufus,Home Manager_Ryoichi,Home Manager_Ryuu,Home Manager_Seiji,Home Manager_Shinichi,Home Manager_Tanjiro,Home Manager_Tarkas/Adelard,Home Manager_Tatsuya,Home Manager_Tenzin,Home Manager_Valkas/Gokin,Home Manager_Yami,Home Manager_Youkai,Home Manager_Zankoku/Garret,Away Manager_Adelard,Away Manager_Adric,Away Manager_Akui,Away Manager_Akuma/Inuyasha,Away Manager_Alex,Away Manager_Alphonse,Away Manager_Andwise/Hob,Away Manager_Aoiro,Away Manager_Ash,Away Manager_Astro,Away Manager_Balbo,Away Manager_Barner,Away Manager_Bill,Away Manager_Bodo/Drogo,Away Manager_Bolt,Away Manager_Brock,Away Manager_Cottar,Away Manager_Daiki,Away Manager_Dante,Away Manager_Devix,Away Manager_Dudo/Eder,Away Manager_Elrick,Away Manager_Frodo,Away Manager_Haldon,Away Manager_Hideaki,Away Manager_Hideo,Away Manager_Hikaru,Away Manager_Hornblower,Away Manager_Isamu,Away Manager_Jiraya,Away Manager_Joyas,Away Manager_Karatachi,Away Manager_Kepli,Away Manager_Kouki,Away Manager_Kouta,Away Manager_Largo,Away Manager_Levi,Away Manager_Masashi,Away Manager_Megumi,Away Manager_Nikushimi,Away Manager_Noburu,Away Manager_Osamu,Away Manager_Otho,Away Manager_Rorimac,Away Manager_Roy,Away Manager_Rufus,Away Manager_Ryoichi,Away Manager_Ryuu,Away Manager_Seiji,Away Manager_Shinichi,Away Manager_Tanjiro,Away Manager_Tarkas/Adelard,Away Manager_Tatsuya,Away Manager_Tenzin,Away Manager_Valkas/Gokin,Away Manager_Yami,Away Manager_Youkai,Away Manager_Zankoku/Garret
0,1.0,2.0,0.0,1.00,1.59,463.0,88.0,5.26,315.0,77.0,4.09,260.0,36.0,7.22,203.0,52.0,3.90,167.0,36.0,4.64,148.0,41.0,3.61,12.0,2.0,6.0,7.0,2361.0,1803.0,1180.0,916.0,1181.0,887.0,145.0,109.0,0.0,1.74,13.20,145.0,109.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### Random Forest

In [119]:
clf = RandomForestClassifier(n_estimators=1000,criterion='entropy', max_depth=25,bootstrap=False,min_samples_leaf= 2,n_jobs=-1)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=25,
                       min_samples_leaf=2, n_estimators=1000, n_jobs=-1)

In [120]:
X_test = pd.DataFrame(np.nan_to_num(X_test),columns=X_test.columns)
y_pred = clf.predict_proba(X_test)
print(log_loss(y_test,y_pred))

0.9214479698862866


In [121]:
y_pred1=clf.predict(X_test)
accuracy_score(y_test,y_pred1)

0.5365853658536586

In [122]:
set(Test.columns)-set(X.columns)

set()

Move on to our whole datset

In [123]:
clf = RandomForestClassifier(n_estimators=180,criterion='entropy', max_depth=12,bootstrap=False,n_jobs=-1)
clf.fit(X, y)
predictions=clf.predict_proba(np.nan_to_num(Test))

In [124]:
col=np.array([s for s in  score_mapping.keys()])
col

array(['Away win', 'Draw', 'Home Win'], dtype='<U8')

In [125]:
test_predictions=predictions
Test[col]=test_predictions

In [126]:
sub=Test[["Game_ID","Away win","Draw","Home Win"]]

In [127]:
game_mapping_inv={v:k for k,v in game_mapping.items()}

In [128]:
sub["Game_ID"]=sub["Game_ID"].apply(lambda x: game_mapping_inv[x])

In [129]:
sub.reset_index(drop=True)

,Game_ID,Away win,Draw,Home Win
0,ID_G91KCS98,0.27,0.30,0.43
1,ID_PYMMJPRC,0.19,0.43,0.39
2,ID_Q0QMD9X9,0.49,0.29,0.23
3,ID_3I1STYJX,0.04,0.16,0.80
4,ID_V8L2BWZV,0.48,0.36,0.15
...,...,...,...,...
229,ID_MWXN4B53,0.19,0.42,0.40
230,ID_QR4DQRG1,0.26,0.42,0.32
231,ID_QKWD8147,0.64,0.28,0.08
232,ID_DLPZZFIJ,0.29,0.35,0.36


In [130]:
sub.to_csv("sub.csv", index=False)

In [131]:
sub.describe()

,Away win,Draw,Home Win
count,234.00,234.00,234.00
mean,0.27,0.37,0.36
std,0.15,0.09,0.17
min,0.02,0.12,0.05
25%,0.16,0.31,0.23
50%,0.26,0.37,0.35
75%,0.36,0.42,0.48
max,0.69,0.66,0.84
